In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [2]:
df = pd.read_csv("data/us_estdata_expanded2.csv")
df

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,PINCP_ORIG_ADJ,HINCP_ORIG_ADJ,SMOCP_ORIG_ADJ,GRNTP_ORIG_ADJ,WAGP_ORIG_ADJ,PERNP_ORIG_ADJ,UNEMP_PCT_ORIG_ADJ,OWN_JOB_ORIG,TYPE_ORIG,CBSA_NAME_ORIG
0,0,P,2018HU0637969,3,1,1104,2,17,1013097,134,...,40000.0,72300.0,944.0,913.0,36800.0,38000.0,0.036364,13058.0,0,218.0
1,1,P,2018HU0808038,5,5,505,3,24,1013097,158,...,51000.0,96400.0,1478.0,1310.0,47000.0,49700.0,0.045407,0.0,0,341.0
2,2,P,2018HU1051188,9,1,7310,4,6,1013097,76,...,43000.0,94200.0,1998.0,1620.0,37000.0,40000.0,0.045469,46774.0,0,13.0
3,3,P,2018HU0461416,2,1,4110,1,36,1013097,60,...,40000.0,90200.0,1840.0,1670.0,38000.0,40000.0,0.051709,18982.0,0,367.0
4,4,P,2018GQ0016994,5,1,51167,3,51,1013097,61,...,45000.0,90050.0,1558.0,1417.0,41150.0,42000.0,0.025789,13230.0,1,477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126540,126540,P,2018HU0922395,9,1,3747,4,6,1013097,174,...,36000.0,83150.0,1933.0,1480.0,31000.0,35000.0,0.051835,91302.0,0,23.0
126541,126541,P,2018HU1191023,8,1,110,4,4,1013097,65,...,41000.0,78800.0,1221.0,1169.0,38000.0,40000.0,0.038677,82767.0,0,53.0
126542,126542,P,2018HU0320162,4,1,500,2,20,1013097,51,...,33000.0,62400.0,909.5,937.0,30000.0,31000.0,0.044170,7506.0,0,238.0
126543,126543,P,2018HU1079519,6,1,300,3,47,1013097,144,...,36000.0,65950.0,990.0,917.5,32000.0,34400.0,0.054773,0.0,1,244.0


In [3]:
df["ORIGIN"].value_counts()

603700     4171
2500390    1959
1703400    1723
400100     1628
4804600    1434
           ... 
1302400      28
2202100      26
5500900      24
2300600      24
800400       21
Name: ORIGIN, Length: 975, dtype: int64

In [4]:
df["MED"] = np.where(df["NAICS"] == "MED", 1, 0)
df["MFG"] = np.where(df["NAICS"] == "MFG", 1, 0)
df["RET"] = np.where(df["NAICS"] == "RET", 1, 0)
df["EDU"] = np.where(df["NAICS"] == "EDU", 1, 0)
df["ADM"] = np.where(df["NAICS"] == "ADM", 1, 0)
df["FOD"] = np.where(df["NAICS"] == "FOD", 1, 0)
df["PRF"] = np.where(df["NAICS"] == "PRF", 1, 0)
df["TRN"] = np.where(df["NAICS"] == "TRN", 1, 0)
df["SRV"] = np.where(df["NAICS"] == "SRV", 1, 0)
df["FIN"] = np.where(df["NAICS"] == "FIN", 1, 0)
df["WHL"] = np.where(df["NAICS"] == "WHL", 1, 0)
df["AGR"] = np.where(df["NAICS"] == "AGR", 1, 0)
df["PUB"] = np.where(df["NAICS"] == "PUB", 1, 0)
df["INF"] = np.where(df["NAICS"] == "INF", 1, 0)
df["ENT"] = np.where(df["NAICS"] == "ENT", 1, 0)
df["REL"] = np.where(df["NAICS"] == "REL", 1, 0)
df["UTL"] = np.where(df["NAICS"] == "UTL", 1, 0)
df["EXT"] = np.where(df["NAICS"] == "EXT", 1, 0)
df["MNG"] = np.where(df["NAICS"] == "MNG", 1, 0)
df["CON"] = np.where(df["NAICS"] == "CON", 1, 0)
df["NO_IND"] = np.where(df["NAICS"].isna(), 1, 0)

In [5]:
df["NO_IND"].sum()

35030

In [6]:
# clean up the database (Biogeme Database can only have numerical values)
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [7]:
df["CHOSEN"].value_counts()

5500100    179
102500     164
5500700    161
5310200    158
1200500    155
          ... 
1208611     20
2701503     20
800400      19
3100904     17
2701403     16
Name: CHOSEN, Length: 2336, dtype: int64

In [8]:
# defining the chosen alterantive for each person explicitly (0 to 35, corresponding to staying and moving to one of the many PUMAs)
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 201): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])
df["CHOSEN"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [9]:
df["CHOSEN"].value_counts()

0    110080
1     16465
Name: CHOSEN, dtype: int64

In [10]:
df["IN_COLLEGE"] = np.where((df["SCHG"] == 15) | (df["SCHG"] == 16), 1, 0)
df["IN_COLLEGE"]

0         0
1         0
2         0
3         0
4         0
         ..
126540    0
126541    0
126542    0
126543    0
126544    0
Name: IN_COLLEGE, Length: 126545, dtype: int32

In [11]:
df["CHOSEN"].value_counts()

0    110080
1     16465
Name: CHOSEN, dtype: int64

In [12]:
df["STAY"].value_counts()

1    110080
0     16465
Name: STAY, dtype: int64

In [13]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [14]:
df["AGE_18_22"] = np.where(df["AGEP"] <= 22, 1, 0)
df["AGE_23_29"] = np.where((df["AGEP"] >= 23) & (df["AGEP"] <= 29), 1, 0)
df["AGE_30_39"] = np.where((df["AGEP"] >= 30) & (df["AGEP"] <= 39), 1, 0)
df["AGE_40_49"] = np.where((df["AGEP"] >= 40) & (df["AGEP"] <= 49), 1, 0)
df["AGE_50_64"] = np.where((df["AGEP"] >= 50) & (df["AGEP"] <= 64), 1, 0)

In [15]:
df["AGEP"].describe()

count    126545.000000
mean         50.029278
std          18.870648
min          18.000000
25%          34.000000
50%          51.000000
75%          64.000000
max          96.000000
Name: AGEP, dtype: float64

In [16]:
df["EDU_LESS_HIGH"] = np.where(df["SCHL"] <= 15, 1, 0)
df["EDU_HIGH"] = np.where((df["SCHL"] <= 20) & (df["SCHL"] >= 16), 1, 0) 
df["EDU_COLLEGE"] = np.where(df["SCHL"] >= 21, 1, 0)
df["EDU_NOCOLLEGE"] = np.where(df["EDU_COLLEGE"] == 0, 1, 0)

In [17]:
df["WOMAN_CHILD"] = np.where((df["PAOC"] >= 1) & (df["PAOC"] <= 3), 1, 0)
df["UNEMPLOYED"] = np.where(df["ESR"] == 3, 1, 0)

In [18]:
df["MALE"] = np.where(df["SEX"] == 1, 1, 0)
df["FEMALE"] = np.where(df["SEX"] == 0, 1, 0)

In [19]:
df["MARRIED"] = np.where(df["MAR"] == 1, 1, 0)

In [20]:
# df["child_old"] = np.where(df["child"] == df["REC_CHILD"], 0, df["child"])
# df["child_old"].value_counts()
df["child"].value_counts()

0    87621
1    38924
Name: child, dtype: int64

In [21]:
df["REC_NO_MAR"] = np.where((df["MARHD"] == 1) | (df["MARHW"] == 1), 1, 0)
df["REC_NO_MAR"].value_counts()

0    124750
1      1795
Name: REC_NO_MAR, dtype: int64

In [22]:
df["MARHM_new"] = np.where(df["MARHM"] == 2, 0, df["MARHM"])
df["MARHM_new"].value_counts()

0.0    124409
1.0      2136
Name: MARHM_new, dtype: int64

In [23]:
df["married_old"] = np.where((df["MARHM"] == df["MARRIED"]), 0, df["MARRIED"])
df["married_old"].value_counts()

1    65916
0    60629
Name: married_old, dtype: int64

In [24]:
df["MILITARY"] = np.where(df["MIL"] == 1, 1, 0)

In [25]:
# df["HH_COST"] = np.where(df["TEN"] == 3, df["GRNTP"], df["SMOCP"])

In [26]:
# up to debate still
df["AGR_EXT"] = np.where((df["AGR"] == 1) | (df["EXT"] == 1), 1, 0)
df["HIGH_ED"] = np.where((df["MED"] == 1) | (df["EDU"] == 1) | (df["PRF"] == 1) | (df["MED"] == 1) | (df["FIN"] == 1) | (df["INF"] == 1) | (df["MED"] == 1), 1, 0)
df["LICENSE"] = np.where((df["SRV"] == 1) | (df["REL"] == 1), 1, 0)
df["OTHER_JOB"] = np.where((df["AGR_EXT"] == 0) & (df["HIGH_ED"] == 0) & (df["LICENSE"] == 0) & (df["NO_IND"] == 0), 1, 0)

In [27]:
df["MICRO_adj_ORIG"] = np.where((df["TYPE_ORIG"] == 3) | (df["TYPE_ORIG"] == 2), 1, 0)
df["METRO"] = np.where(df["TYPE_ORIG"] == 1, 1, 0)
df["T34"] = np.where(df["TYPE_ORIG"] == 0, 1, 0)

In [28]:
for i in range(1, 201):
    key = "ALT{0}_".format(i)
    df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
    df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
    df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

...

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result o

In [29]:
df["UNEMPLOYED"] = np.where((df["ESR"] == 3) | (df["ESR"] == 6), 1, 0)
df["IN_LF"] = np.where(df["ESR"] == 6, 0, 1)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\2921326413.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["IN_LF"] = np.where(df["ESR"] == 6, 0, 1)


In [30]:
df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
df["OTHER_FAMILY"] = np.where((df["HHT"] == 2) | (df["HHT"] == 3), 1, 0)

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3935644085.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3935644085.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
C:\Users\Transportlab\AppData\Local\Temp\ipykernel_7348\3935644085.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [31]:
df.loc[df["CBSA_NAME_ORIG"] == -1, "CBSA_NAME_ORIG"] = -2

In [32]:
# df["INTERNAL"] = df["ORIGIN"] == df["CHOSEN"]

In [33]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df)

In [34]:
# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [35]:
c_move = Beta("c_move", -0.1706511, None, None, 0)

In [36]:
df["ALT3_PUMA"]

0         2400400
1         1209909
2         3701803
3          607307
4         4201200
           ...   
126540    4001003
126541     608506
126542    3904110
126543    3400904
126544    4801502
Name: ALT3_PUMA, Length: 126545, dtype: int64

In [37]:
df["ALT103_PUMA"]

0         2400301
1         1300500
2          800822
3         3904400
4          607112
           ...   
126540     608511
126541    3603206
126542    3603804
126543     603738
126544     900305
Name: ALT103_PUMA, Length: 126545, dtype: int64

In [38]:
# Staying Choice Parameters to be Estimated
c_stay_age_18_22 = Beta("c_stay_age_18_22", -0.3518474, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", -0.3122236, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", -0.1526367, None, None, 0)
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0.2089962, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0.9886252, None, None, 0)
c_stay_child = Beta("c_stay_child", -0.2019415, None, None, 0)
c_stay = Beta("c_stay", 15.83822, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0.3598987, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0.00000437, None, None, 0)
c_stay_college = Beta("c_stay_college", 0.4184854, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", 0.1389506, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", -1.018068, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", -0.4515589, None, None, 0)
c_stay_mil = Beta("c_stay_mil", -0.8357647, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 0.6723992, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0.4348394, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0.2881474, None, None, 0)
c_stay_ownjob = Beta("c_stay_ownjob", 1.559779, None, None, 0)
c_stay_rentcost = Beta("c_stay_rentcost", -0.0008114205, None, None, 0)
c_stay_owncost = Beta("c_stay_owncost", -0.0001282814, None, None, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0.000001735451, None, None, 0)
c_stay_income = Beta("c_stay_income", 0.00002259662, 0, None, 0)

In [39]:
c_stay_T34 = Beta("c_stay_T34", 0, None, None, 0)
c_stay_metro = Beta("c_stay_metro", 0, None, None, 0)
c_stay_micro = Beta("c_stay_micro", 0, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", -0.4674334, None, None, 0)
c_stay_edu_high = Beta("c_stay_edu_high", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0.1799824, None, None, 0)

In [40]:
# # defining the staying utility function
# V0 = c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_child * child + c_stay + c_stay_metro * METRO + c_stay_micro * MICRO_adj_ORIG + c_stay_college * IN_COLLEGE + c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR + c_stay_rec_child * REC_CHILD + c_stay_mil * MILITARY + c_stay_hh_val * HH_MED_VAL_ORIG + MARRIED * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY + c_stay_ownjob * OWN_JOB_ORIG / TOT_JOBS_ORIG + (c_stay_rentcost * GRNTP_ORIG_ADJ + c_stay_owncost * SMOCP_ORIG_ADJ) / HINCP_ORIG_ADJ * 12 + c_stay_foreign * FOREIGN + c_stay_edu_college * EDU_COLLEGE + c_stay_edu_high * EDU_HIGH

In [41]:
# defining the staying utility function
V0 = c_stay + \
c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 \
+ c_stay_child * child + c_stay_rec_child * REC_CHILD \
+ c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR \
+ MARRIED * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY \
+ c_stay_college * IN_COLLEGE + c_stay_mil * MILITARY + c_stay_ownjob * OWN_JOB_ORIG / TOT_JOBS_ORIG \
+ c_stay_hh_val * HH_MED_VAL_ORIG + c_stay_rentcost * GRNTP_ORIG_ADJ + c_stay_owncost * SMOCP_ORIG_ADJ \
+ c_stay_foreign * FOREIGN \
+ c_stay_edu_college * EDU_COLLEGE + c_stay_edu_nohigh * EDU_LESS_HIGH \
+ IN_LF * (c_stay_income * PERNP_ORIG_ADJ + c_stay_unemp_rate * UNEMP_PCT_ORIG_ADJ) \
+ c_stay_T34 * T34 + c_stay_micro * MICRO_adj_ORIG

In [42]:
df["PINCP_ORIG_ADJ"]

0         40000.0
1         51000.0
2         43000.0
3         40000.0
4         45000.0
           ...   
126540    36000.0
126541    41000.0
126542    33000.0
126543    36000.0
126544    39885.0
Name: PINCP_ORIG_ADJ, Length: 126545, dtype: float64

In [43]:
print(V0)

((((((((((((((((((((((((c_stay(15.83822) + (c_stay_age_18_22(-0.3518474) * AGE_18_22)) + (c_stay_age_23_29(-0.3122236) * AGE_23_29)) + (c_stay_age_30_39(-0.1526367) * AGE_30_39)) + (c_stay_age_40_49(0) * AGE_40_49)) + (c_stay_age_50_64(0.2089962) * AGE_50_64)) + (c_stay_age_65(0.9886252) * AGE_OVER_65)) + (c_stay_child(-0.2019415) * child)) + (c_stay_rec_child(0.1389506) * REC_CHILD)) + (c_stay_rec_mar(-1.018068) * MARHM_new)) + (c_stay_rec_nomar(-0.4515589) * REC_NO_MAR)) + (MARRIED * ((c_stay_2work_mar(0.6723992) * WORK2_MAR) + (c_stay_1work_mar(0.4348394) * WORK1_MAR)))) + (c_stay_otherfamily(0.2881474) * OTHER_FAMILY)) + (c_stay_college(0.4184854) * IN_COLLEGE)) + (c_stay_mil(-0.8357647) * MILITARY)) + ((c_stay_ownjob(1.559779) * OWN_JOB_ORIG) / TOT_JOBS_ORIG)) + (c_stay_hh_val(1.735451e-06) * HH_MED_VAL_ORIG)) + (c_stay_rentcost(-0.0008114205) * GRNTP_ORIG_ADJ)) + (c_stay_owncost(-0.0001282814) * SMOCP_ORIG_ADJ)) + (c_stay_foreign(0.3598987) * FOREIGN)) + (c_stay_edu_college(-0.46

In [44]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_destchoice_dist=Beta('c_destchoice_dist', -0.00000005961744, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', -0.2172545, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp_rate", -2.147723, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", -2.145727, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0.00005504696, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 5.330245, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 2.187570, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 7.440407, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 2.149015, None, None, 0)
c_destchoice_pincp = Beta("c_destchoice_pincp", 0, 0, None, 0)
c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 3.594148, None, None, 0)
c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64",3.530389, None, None, 0)
c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 3.861166, None, None, 0)
c_destchoice_military = Beta("c_destchoice_military", 0.0005411581, None, None, 0)

In [45]:
c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0.2292714, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0.1630156, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0.05934898, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0.6677951, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 0.7053572, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", -0.3005452, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0.5022687, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 1.19394, None, None, 0)

In [46]:
c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 7.446647, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 2.159228, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 6.702034, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 2.617076, None, None, 0)

In [47]:
c_destchoice_samestate = Beta("c_destchoice_samestate", 6.342653, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 1.216911, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 0.3296348, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", -0.00001927862, None, None, 0)

In [48]:
# birthstate > cbsa > state

In [49]:
# defining the utility functions for each of the moving PUMA alternatives
# defined using the exec to parse a string to save space
# can also use a loop to print out the statements and then copy/paste them to run
# can also just write each one manually
for i in range(200):
    num = i + 1
    initialization = "V{0} = c_move + log(ALT{0}_POP) + (CBSA_NAME_ORIG != ALT{0}_CBSA) * (c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1)) + (CBSA_NAME_ORIG == ALT{0}_CBSA) * c_destchoice_cbsa_dist * ALT{0}_DIST + \
        \
    c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + \
        \
    c_destchoice_age_18_34 * AGE_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * AGE_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * AGE_OVER_65 * ALT{0}_65 / ALT{0}_POP + \
        \
    ALT{0}_ENT / (ALT{0}_EMP + 1) * (c_destchoice_entscore_18_34 * AGE_18_34 + c_destchoice_entscore_35_64 * AGE_35_64 + c_destchoice_entscore_65 * AGE_OVER_65) + \
        \
    IN_LF * (c_destchoice_unemp_rate * ALT{0}_UNEMP_PCT + c_destchoice_pincp * ALT{0}_PERNP) + \
        \
    (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
    (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
    (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
        \
    (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER_JOB) * ALT{0}_OWN_JOB / (ALT{0}_EMP + 1) + c_destchoice_military * MILITARY * ALT{0}_MIL_NUM + \
        \
    c_destchoice_birthstate * (POBP == ALT{0}_State) + c_destchoice_samecbsa * (CBSA_NAME_ORIG == ALT{0}_CBSA) + c_destchoice_samestate * (MIGSP == ALT{0}_State)".format(num)
    exec(initialization)
print(V5)

# full model specification (takes a bit longer to run):
# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_entscore * ALT{0}_ENT / AGEP / ALT{0}_EMP + c_destchoice_unemp * ALT{0}_UNEMP + c_destchoice_pincp * ALT{0}_PINCP + \
# (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
# (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
# (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
# (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER) * ALT{0}_OWN_JOB / ALT{0}_EMP".format(num)

# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_adm * ADM * ALT{0}_ADM/ALT{0}_EMP + c_agr * AGR * ALT{0}_AGR/ALT{0}_EMP + c_con * CON * ALT{0}_CON/ALT{0}_EMP + c_edu * EDU * ALT{0}_EDU/ALT{0}_EMP + c_ent * ENT * ALT{0}_ENT/ALT{0}_EMP + c_ext * EXT * ALT{0}_EXT/ALT{0}_EMP + c_fin * FIN * ALT{0}_FIN/ALT{0}_EMP + c_fod * FOD * ALT{0}_FOD/ALT{0}_EMP + c_inf * INF * ALT{0}_INF/ALT{0}_EMP + c_med * MED * ALT{0}_MED/ALT{0}_EMP + c_mfg * MFG * ALT{0}_MFG/ALT{0}_EMP + c_mng * MNG * ALT{0}_MNG/ALT{0}_EMP + c_prf * PRF * ALT{0}_PRF/ALT{0}_EMP + c_pub * PUB * ALT{0}_PUB/ALT{0}_EMP + c_rel * REL * ALT{0}_REL/ALT{0}_EMP + c_ret * RET * ALT{0}_RET/ALT{0}_EMP + c_srv * SRV * ALT{0}_SRV/ALT{0}_EMP + c_trn * TRN * ALT{0}_TRN/ALT{0}_EMP + c_utl * UTL * ALT{0}_UTL/ALT{0}_EMP + c_whl * WHL * ALT{0}_WHL/ALT{0}_EMP"

# for the fields already in the Biogeme db.Database, can explicitly refer to them; also used a few references to other databases using .loc and fields in the Biogeme database

(((((((((((((((((((c_move(-0.1706511) + log(ALT5_POP)) + ((CBSA_NAME_ORIG != ALT5_CBSA) * ((c_destchoice_dist(-5.961744e-08) * ALT5_DIST) + (c_destchoice_logdist(-0.2172545) * log((ALT5_DIST + `1`)))))) + (((CBSA_NAME_ORIG == ALT5_CBSA) * c_destchoice_cbsa_dist(-1.927862e-05)) * ALT5_DIST)) + (c_destchoice_hhcost(-2.145727) * ALT5_HH_COST)) + ((c_destchoice_college(5.504696e-05) * IN_COLLEGE) * ALT5_COLLEGE)) + (((c_destchoice_foreign(2.149015) * FOREIGN) * ALT5_FOREIGN) / ALT5_POP)) + (((c_destchoice_age_18_34(5.330245) * AGE_18_34) * ALT5_18_34) / ALT5_POP)) + (((c_destchoice_age_35_64(2.18757) * AGE_35_64) * ALT5_35_64) / ALT5_POP)) + (((c_destchoice_age_over_65(7.440407) * AGE_OVER_65) * ALT5_65) / ALT5_POP)) + ((ALT5_ENT / (ALT5_EMP + `1`)) * (((c_destchoice_entscore_18_34(3.594148) * AGE_18_34) + (c_destchoice_entscore_35_64(3.530389) * AGE_35_64)) + (c_destchoice_entscore_65(3.861166) * AGE_OVER_65)))) + (IN_LF * ((c_destchoice_unemp_rate(-2.147723) * ALT5_UNEMP_PCT) + (c_destch

In [50]:
utilities = {}
for i in range(126):
    init = "utilities[{0}] = V{0}".format(i)
    exec(init)

In [51]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-07-24]\n'

In [52]:
# move = Beta("move", 1.497034, 1, None, 0)

In [53]:
# nest_move = move, list(range(1, 151))
# nest_stay = 1.0, [0]
# nests = nest_move, nest_stay

In [54]:
# V = utilities

# av = {}
# for i in range(151):
#     av[i] = 1

# nest_logprob = models.lognested(V, av, nests, CHOSEN)

# biogeme_nest = bio.BIOGEME(database, nest_logprob, suggestScales=False)
# biogeme_nest.modelName = "nested_full_full"

# biogeme_nest.calculateNullLoglikelihood(av)

# results_nest = biogeme_nest.estimate()
# pandasResults_nest = results_nest.getEstimatedParameters()
# print(pandaResults_nest)

In [55]:
# Associate utility functions with the numbering of alternatives (corresponds to the CHOSEN field created earlier)
V = utilities

# Associate the availability conditions with the alternatives
# for this model, all migrants had all alternatives theoretically available so all are equal to 1 (available)
# if individual people had different availability for alterantives, could pass in a column of the dataframe to account for that availability
av = {}
for i in range(126):
    av[i] = 1

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, av, CHOSEN)

# formulas = {"loglike": logprob, "weight": W0}

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, suggestScales=False, skipAudit=True)
biogeme.modelName = 'us_full_final'

biogeme.saveIterations = False

[20:16:23] < General >   Remove 2469 unused variables from the database as only 2415 are used.


In [56]:
# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)

-612007.2939151749

In [57]:
#estimate parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[20:21:21] < Detailed >  Log likelihood (N = 126545):  -60929.76
[20:21:21] < Detailed >  ** Optimization: Newton with trust region for simple bounds
[21:35:46] < General >   Log likelihood (N = 126545):  -60929.76 Gradient norm:      2e+10 Hessian norm:       6e+16 
[21:40:40] < Detailed >  Log likelihood (N = 126545):  -45555.91
[22:48:39] < General >   Log likelihood (N = 126545):  -45555.91 Gradient norm:      6e+09 Hessian norm:       3e+16 
[22:48:39] < Detailed >  1 f= 0.3599977 projected rel. grad.=4.7e+04 rel. change=  0.34 delta=     2 rho=   1.1 ++
[22:53:45] < Detailed >  Log likelihood (N = 126545):  -43963.96
[23:56:02] < General >   Log likelihood (N = 126545):  -43963.96 Gradient norm:      2e+09 Hessian norm:       1e+16 
[23:56:02] < Detailed >  2 f= 0.3474176 projected rel. grad.=1.4e+04 rel. change=  0.98 delta=     2 rho=  0.83 +
[00:01:05] < Detailed >  Log likelihood (N = 126545):  -43515.91
[00:59:38] < General >   Log likelihood (N = 126545):  -43515.91 Gradien

In [ ]:
# occupation-based coefficients

c_adm = Beta("c_adm", 0, None, None, 0)
c_agr = Beta("c_agr", 0, None, None, 0)
c_con = Beta("c_con", 0, None, None, 0)
c_edu = Beta("c_edu", 0, None, None, 0)
c_ent = Beta("c_ent", 0, None, None, 0)
c_ext = Beta("c_ext", 0, None, None, 0)
c_fin = Beta("c_fin", 0, None, None, 0)
c_fod = Beta("c_fod", 0, None, None, 0)
c_inf = Beta("c_inf", 0, None, None, 0)
c_med = Beta("c_med", 0, None, None, 0)
c_mfg = Beta("c_mfg", 0, None, None, 0)
c_mng = Beta("c_mng", 0, None, None, 0)
c_prf = Beta("c_prf", 0, None, None, 0)
c_pub = Beta("c_pub", 0, None, None, 0)
c_rel = Beta("c_rel", 0, None, None, 0)
c_ret = Beta("c_ret", 0, None, None, 0)
c_srv = Beta("c_srv", 0, None, None, 0)
c_trn = Beta("c_trn", 0, None, None, 0)
c_utl = Beta("c_utl", 0, None, None, 0)
c_whl = Beta("c_whl", 0, None, None, 0)

In [ ]:
# Staying Choice Parameters with starting values of 0
c_stay_married = Beta("c_stay_married", 0, None, None, 0)
c_stay_age_18_22 = Beta("c_stay_age_18_22", 0, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", 0, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", 0, None, None, 0)
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", 0, None, None, 0)
c_stay_edu_beyondhs = Beta('c_stay_edu_beyondhs', 0, None, None, 0)
c_stay_child = Beta("c_stay_child", 0, None, None, 0)
c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
c_stay = Beta("c_stay", 15, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
c_stay_college = Beta("c_stay_college", 0, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", 0, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", 0, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", 0, None, None, 0)
c_stay_mil = Beta("c_stay_mil", 0, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 0, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0, None, None, 0)
c_stay_income = Beta("c_stay_income", 0, None, None, 0)
c_stay_hhinc = Beta("c_stay_hhinc", 0, None, None, 0)
c_stay_hurent = Beta('c_stay_hurent', 0, None, None, 0)
c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
c_stay_hhcost = Beta("c_stay_hhcost", 0, None, None, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, None, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0, None, None, 0)
c_stay_partcp = Beta("c_stay_partcp", 0, None, None, 0)
c_stay_rentcost = Beta("c_stay_rentcost", 0, None, None, 0)
c_stay_owncost = Beta("c_stay_owncost", 0, None, None, 0)

In [ ]:
# Bounded staying choice parameters
c_stay_income = Beta("c_stay_income", 0, 0, None, 0)
c_stay_hhinc = Beta("c_stay_hhinc", 0, 0, None, 0)
c_stay_hurent = Beta('c_stay_hurent', 0, None, 0, 0)
c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
c_stay_hhcost = Beta("c_stay_hhcost", 0, None, 0, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0, 0, None, 0)

In [ ]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_destchoice_dist=Beta('c_destchoice_dist', 0, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', 0, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp", 0, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", 0, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 0, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 0, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
c_destchoice_pincp = Beta("c_destchoice_pincp", 0, None, None, 0)

c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 0, None, None, 0)
c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 0, None, None, 0)
c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 0, None, None, 0)

c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 0, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", 0, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 0, None, None, 0)

c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 0, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 0, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 0, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 0, None, None, 0)

c_destchoice_samestate = Beta("c_destchoice_samestate", 0, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 0, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", 0, None, None, 0)
c_destchoice_military = Beta("c_destchoice_military", 0, None, None, 0)